# How to Transform Time Series to a Supervised Learning Problem

Time series forecasting can be framed as a supervised learning problem. This re-framing of your
time series data allows you access to the suite of standard linear and nonlinear machine learning
algorithms on your problem. 

In [1]:
import numpy as np

In [2]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [3]:
s = np.sin(np.linspace(0,5))
X, y = split_sequence(s, 4)

In [9]:
# Define univariate time series
series = np.array([i+1 for i in range(10)])
print(series.shape)

# Transform into a supervised learning problem
X, y = split_sequence(series, 3)
print(X.shape, y.shape)

# Transform input from [samples, features] to [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))
print(X.shape)

(10,)
(7, 3) (7,)
(7, 3, 1)


## Data Preparation Example

Consider that you are in the current situation:
```
I have two columns in my data file with 5,000 rows, column 1 is time (with 1 hour
interval) and column 2 is the number of sales and I am trying to forecast the number
of sales for future time steps. Help me to set the number of samples, time steps and
features in this data for an LSTM?
```
There are few problems here:
* Data Shape. LSTMs expect 3D input, and it can be challenging to get your head around this the first time.
* Sequence Length. LSTMs don’t like sequences of more than 200-400 time steps, so the data will need to be split into subsamples

In [11]:
# define the dataset
data = list()
n = 5000
for i in range(n):
    data.append([i+1, (i+1)*10])
data = np.array(data)
print(data[:5, :])
print(data.shape)

[[ 1 10]
 [ 2 20]
 [ 3 30]
 [ 4 40]
 [ 5 50]]
(5000, 2)


In [12]:
# drop time
data = data[:, 1]
print(data.shape)

(5000,)


## Split Into Samples
LSTMs need to process samples where each sample is a single sequence of observations. In this case, 5,000 time steps is too long; LSTMs work better with 200-to-400 time steps. Therefore, we need to split the 5,000 time steps into multiple shorter sub-sequences. There are many ways to do this, and you may want to explore some depending on your problem. For example, perhaps
you need overlapping sequences, perhaps non-overlapping is good but your model needs state across the sub-sequences and so on. In this example, we will split the 5,000 time steps into 25 sub-sequences of 200 time steps each. Rather than using NumPy or Python tricks, we will do this the old fashioned way so you can see what is going on.

In [15]:
# example of splitting a univariate sequence into subsequence
from numpy import array

# define the dataset
data = list()
n = 5000
for i in range(n):
    data.append([i+1, (i+1)*10])
data = array(data)

# drop time
data = data[:, 1]

# split into samples (e.g. 5000/200 = 25)
samples = list()
length = 200

# step over the 5000 in jumps of 200
for i in range(0, n, length):
    # grab from i to 1 + 200
    sample = data[i:i+length]
    samples.append(sample)
print(len(samples))

25


## Reshape Subsequences
The LSTM needs data with the format of [samples, timesteps, features]. We have 25 samples, 200 time steps per sample, and 1 feature. First, we need to convert our list of arrays into a 2D NumPy array with the shape [25, 200].

In [21]:
# convert list of arrays into 2d array
data = array(samples)
data.shape

(25, 200)

In [22]:
# reshape into [samples, timesteps, features]
data = data.reshape((len(samples), length, 1))
data.shape

(25, 200, 1)